In [25]:
import gurobipy as gp
from gurobipy import GRB


In [26]:
commodities = ['Pencils', 'Pens']
nodes = ['Detroit', 'Denver', 'Boston', 'New York', 'Seattle']

In [27]:
arcs, capacity = gp.multidict({
    ('Detroit', 'Boston'):   100,
    ('Detroit', 'New York'):  80,
    ('Detroit', 'Seattle'):  120,
    ('Denver',  'Boston'):   120,
    ('Denver',  'New York'): 120,
    ('Denver',  'Seattle'):  120})

# Cost for triplets commodity-source-destination
cost = {
    ('Pencils', 'Detroit', 'Boston'):   10,
    ('Pencils', 'Detroit', 'New York'): 20,
    ('Pencils', 'Detroit', 'Seattle'):  60,
    ('Pencils', 'Denver',  'Boston'):   40,
    ('Pencils', 'Denver',  'New York'): 40,
    ('Pencils', 'Denver',  'Seattle'):  30,
    ('Pens',    'Detroit', 'Boston'):   20,
    ('Pens',    'Detroit', 'New York'): 20,
    ('Pens',    'Detroit', 'Seattle'):  80,
    ('Pens',    'Denver',  'Boston'):   60,
    ('Pens',    'Denver',  'New York'): 70,
    ('Pens',    'Denver',  'Seattle'):  30}

# Supply (> 0) and demand (< 0) for pairs of commodity-city
inflow = {
    ('Pencils', 'Detroit'):   50,
    ('Pencils', 'Denver'):    60,
    ('Pencils', 'Boston'):   -50,
    ('Pencils', 'New York'): -50,
    ('Pencils', 'Seattle'):  -10,
    ('Pens',    'Detroit'):   60,
    ('Pens',    'Denver'):    40,
    ('Pens',    'Boston'):   -40,
    ('Pens',    'New York'): -30,
    ('Pens',    'Seattle'):  -30}


In [28]:
m = gp.Model('netflow')

# Create variables
flow = m.addVars(commodities, arcs, obj=cost, name="flow")

# Arc-capacity constraints
m.addConstrs(
    (flow.sum('*', i, j) <= capacity[i, j] for i, j in arcs), "cap")


{('Detroit', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

In [29]:
m.addConstrs(
    (flow.sum(h, '*', j) + inflow[h, j] == flow.sum(h, j, '*')
        for h in commodities for j in nodes), "node")

m.write('pensil.lp')

In [30]:
times = []

In [31]:
for j in range (6):
    m.Params.Method = j
    time = []
    for i in range(100):
        m.optimize()
        time.append(m.Runtime)
        m.reset()
    times.append(time)

Set parameter Method to value 0
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 12 columns and 36 nonzeros
Model fingerprint: 0xc43e5943
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   2.000000e+01      0s
Extra simplex iterations after uncrush: 1
       1    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.500000000e+03
Discarded solution information
Gurobi Optimize

In [32]:
import numpy as np

In [33]:
dict_methods = {0: "primal simplex", 1: "dual simplex", 2: "barrier",3: "concurrent", 4: "deterministic concurrent", 5: "deterministic concurrent simplex"}

In [34]:
for i in range(6):
    print(f'for method \"{dict_methods.get(i)}\":\nmean value: {np.mean(np.array(times[i]))}\nvarience: {np.var(np.array(times[i]))}')

for method "primal simplex":
mean value: 0.01568288803100586
varience: 4.2744630734432574e-05
for method "dual simplex":
mean value: 0.012688565254211425
varience: 4.331281356201088e-05
for method "barrier":
mean value: 0.008300018310546876
varience: 1.293482912274158e-05
for method "concurrent":
mean value: 0.01025907039642334
varience: 3.7820090234640696e-05
for method "deterministic concurrent":
mean value: 0.008864920139312744
varience: 1.2988784618590897e-05
for method "deterministic concurrent simplex":
mean value: 0.009059867858886718
varience: 1.3668210178502701e-05
